<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=950dfbc8b7f113990609506df9af9dbe883a3607b85af937114082ef09f7d05e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 12:27:13
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 20.44 K (0.14%)
Current PnL: -17.40 L (-11.71%)
CY Booked + Current PnL: -6.37 L (-4.29%)
-------------------
Total profit:  2.09 L
Total loss:  -19.50 L
-------------------
Total Booked + Current PnL: 20.61 L (16.93%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.51 L (58.89%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SBIN,863.00,-11.34,61.0,M-LC,4.25,218532.0,17052.0,10162.0,0.13,8.46,4.65,13.51,60.0,1.68,1.59,22.13,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.97,56.0,H-LC,11.12,171831.0,15654.0,10447.0,-0.81,10.02,6.08,16.71,22.0,1.50,1.25,29.21,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,104.70,59.0,M-SC,4.71,88163.0,-12614.0,12704.0,0.45,-12.52,14.41,0.09,245.0,-0.99,0.64,15.83,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.12,116680.0,16176.0,14468.0,0.85,16.09,12.40,30.49,79.0,1.12,0.85,53.71,MH,ATH,METALS
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201664.0,1526.0,19360.0,-0.33,0.76,9.60,10.44,4.0,0.08,1.47,6.37,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.77,44.0,H-MC,2.19,136255.0,467.0,42280.0,0.35,0.34,31.03,31.48,107.0,0.01,0.99,12.50,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.20,61.0,H-MC,1.78,248321.0,590.0,74471.0,1.22,0.24,29.99,30.30,92.0,0.01,1.80,13.56,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.74,180653.0,845.0,111571.0,0.10,0.47,61.76,62.52,88.0,0.01,1.31,34.72,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.28,39.0,H-LC,5.80,301614.0,-14228.0,126889.0,0.63,-4.50,42.07,35.67,5.0,-0.11,2.19,8.73,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.56,43.0,H-LC,13.42,267460.0,95.0,65367.0,-0.06,0.04,24.44,24.48,48.0,0.00,1.94,55.03,XY24,BTT,FINANCE
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.13,229770.0,-12154.0,75433.0,-0.25,-5.02,32.83,26.15,8.0,-0.16,1.67,8.58,X40,ATH,IT
6,ASIANPAINT,4250.00,-12.54,49.0,H-LC,8.42,220915.0,-30853.0,153094.0,-1.41,-12.25,69.30,48.55,27.0,-0.20,1.61,18.83,X40,BTT,PAINTS
80,UNITDSPR,1644.00,-13.66,49.0,H-LC,8.00,226490.0,-7654.0,56283.0,0.54,-3.27,24.85,20.77,86.0,-0.14,1.65,3.42,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-2.89,61.0,H-LC,1.09,240480.0,-5671.0,84889.0,0.22,-2.30,35.30,32.18,16.0,-0.07,1.75,32.91,X200,ATH,IT
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201664.0,1526.0,19360.0,-0.33,0.76,9.60,10.44,4.0,0.08,1.47,6.37,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,61.0,H-LC,4.14,284708.0,19282.0,37496.0,-0.07,7.26,13.17,21.40,11.0,0.51,2.07,15.61,XY25,NTT,FMCG
83,VBL,671.64,-16.28,39.0,H-LC,5.80,301614.0,-14228.0,126889.0,0.63,-4.50,42.07,35.67,5.0,-0.11,2.19,8.73,X40N,ATH,FMCG
56,RELIANCE,1533.00,-12.91,53.0,H-LC,5.86,217651.0,7285.0,21504.0,0.01,3.46,9.88,13.68,37.0,0.34,1.58,20.56,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,7969.85,2.31,62.0,H-LC,28.54,164500.0,-21595.0,233985.0,2.75,-11.60,142.24,114.13,15.0,-0.09,1.20,21.14,AR,ATH,ELECTRICAL
75,TITAGARH,1548.00,1.40,71.0,H-SC,4.95,166381.0,-25634.0,104521.0,2.51,-13.35,62.82,41.08,158.0,-0.25,1.21,40.99,XY24,NTT,ENGINEERING
1,ABB,7934.00,-38.31,64.0,H-LC,11.50,257698.0,-3921.0,123128.0,2.29,-1.50,47.78,45.57,7.0,-0.03,1.87,9.65,AR,NTT,ELECTRICAL
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,13.69,89105.0,-11858.0,106391.0,2.05,-11.75,119.40,93.63,148.0,-0.11,0.65,32.72,SR,ATH,CHEMICALS
53,RAJESHEXPO,518.00,1858.33,64.0,L-SC,4.96,53440.0,-83737.0,83831.0,1.89,-61.04,156.87,0.07,267.0,-1.00,0.39,32.91,OX40N,NTT,JEWELLERY


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-41.65,53.0,L-SC,45.56,85536.0,-28013.0,68061.0,-1.55,-24.67,79.57,35.27,268.0,-0.41,0.62,109.35,XR,NTT,HOTELS
6,ASIANPAINT,4250.0,-12.54,49.0,H-LC,8.42,220915.0,-30853.0,153094.0,-1.41,-12.25,69.30,48.55,27.0,-0.20,1.61,18.83,X40,BTT,PAINTS
85,WHIRLPOOL,2270.0,-40.79,51.0,M-SC,1.11,100148.0,8650.0,70104.0,-1.20,9.45,70.00,86.07,223.0,0.12,0.73,45.43,XR,NTT,DURABLES
46,LAOPALA,464.0,111.57,47.0,H-SC,2.29,96690.0,-34415.0,85658.0,-1.16,-26.25,88.59,39.09,142.0,-0.40,0.70,24.53,AR,NTT,CERAMICS
7,ASIANTILES,137.0,6527.78,42.0,L-SC,5.69,72724.0,-21086.0,97705.0,-1.15,-22.48,134.35,81.67,269.0,-0.22,0.53,40.83,XR,NTT,CERAMICS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,61.0,H-SC,12.48,126738.0,-15741.0,35259.0,0.69,-11.05,27.82,13.70,163.0,-0.45,0.92,49.59,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,59.0,M-SC,4.71,88163.0,-12614.0,12704.0,0.45,-12.52,14.41,0.09,245.0,-0.99,0.64,15.83,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,61.0,H-SC,1.77,224559.0,-45108.0,81448.0,0.80,-16.73,36.27,13.47,138.0,-0.55,1.63,13.58,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,2.16,145153.0,-30750.0,72765.0,-0.17,-17.48,50.13,23.89,149.0,-0.42,1.05,25.36,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.85,108205.0,-23630.0,65854.0,-0.46,-17.92,60.86,32.03,98.0,-0.36,0.79,21.20,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,51.0,H-LC,13.11,292227.0,-53729.0,177586.0,-0.79,-15.53,60.77,35.80,1.0,-0.30,2.12,3.52,X40,BTT,IT
39,INFY,2275.00,-17.44,53.0,H-LC,9.11,322099.0,9211.0,162467.0,-0.88,2.94,50.44,54.87,3.0,0.06,2.34,9.79,X40,BTT,IT
41,ITC,452.00,-37.50,53.0,H-LC,2.91,201664.0,1526.0,19360.0,-0.33,0.76,9.60,10.44,4.0,0.08,1.47,6.37,X40,NTT,FMCG
83,VBL,671.64,-16.28,39.0,H-LC,5.80,301614.0,-14228.0,126889.0,0.63,-4.50,42.07,35.67,5.0,-0.11,2.19,8.73,X40N,ATH,FMCG
1,ABB,7934.00,-38.31,64.0,H-LC,11.50,257698.0,-3921.0,123128.0,2.29,-1.50,47.78,45.57,7.0,-0.03,1.87,9.65,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,66.0,M-MC,6.95,229026.0,4064.0,163593.0,0.56,1.81,71.43,74.53,192.0,0.02,1.66,34.64,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.02,52.0,H-MC,3.81,188140.0,23470.0,44495.0,-0.19,14.25,23.65,41.28,89.0,0.53,1.37,23.37,X40N,ATH,AC
84,VOLTAS,1918.49,1.02,58.0,H-MC,2.83,211860.0,20118.0,75909.0,0.63,10.49,35.83,50.08,99.0,0.27,1.54,18.00,XY25,ATH,AC
30,HAVELLS,2069.16,0.20,61.0,H-MC,1.78,248321.0,590.0,74471.0,1.22,0.24,29.99,30.30,92.0,0.01,1.80,13.56,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.30,57.0,H-MC,6.38,210619.0,13591.0,76033.0,0.22,6.90,36.10,45.49,102.0,0.18,1.53,20.84,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.12,116680.0,16176.0,14468.0,0.85,16.09,12.40,30.49,79.0,1.12,0.85,53.71,MH,ATH,METALS
36,INDIAMART,4850.92,-50.53,54.0,H-SC,12.22,138107.0,14771.0,118993.0,0.20,11.98,86.16,108.45,119.0,0.12,1.00,37.01,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,1.11,100148.0,8650.0,70104.0,-1.20,9.45,70.00,86.07,223.0,0.12,0.73,45.43,XR,NTT,DURABLES
86,WIPRO,420.00,-11.07,54.0,M-LC,5.81,156203.0,5258.0,104203.0,-0.78,3.48,66.71,72.51,53.0,0.05,1.14,9.72,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.74,180653.0,845.0,111571.0,0.10,0.47,61.76,62.52,88.0,0.01,1.31,34.72,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.56,85536.0,-28013.0,68061.0,-1.55,-24.67,79.57,35.27,268.0,-0.41,0.62,109.35,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,13.69,89105.0,-11858.0,106391.0,2.05,-11.75,119.40,93.63,148.0,-0.11,0.65,32.72,SR,ATH,CHEMICALS
48,MASFIN,398.61,-15.09,61.0,H-SC,12.86,96495.0,-1485.0,23091.0,-0.34,-1.52,23.93,22.05,152.0,-0.06,0.70,40.26,XR,ATH,FINANCE
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.12,116680.0,16176.0,14468.0,0.85,16.09,12.40,30.49,79.0,1.12,0.85,53.71,MH,ATH,METALS
8,ATULAUTO,844.00,3845.95,64.0,M-SC,6.74,122956.0,-24412.0,76233.0,-0.71,-16.57,62.00,35.16,236.0,-0.32,0.89,25.59,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3845.95,64.0,M-SC,6.74,122956.0,-24412.0,76233.0,-0.71,-16.57,62.00,35.16,236.0,-0.32,0.89,25.59,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.56,85536.0,-28013.0,68061.0,-1.55,-24.67,79.57,35.27,268.0,-0.41,0.62,109.35,XR,NTT,HOTELS
49,NATIONALUM,247.44,-41.18,75.0,H-MC,5.12,116680.0,16176.0,14468.0,0.85,16.09,12.40,30.49,79.0,1.12,0.85,53.71,MH,ATH,METALS
27,GREENPANEL,537.00,239.79,68.0,M-SC,4.70,151719.0,-31359.0,110330.0,-0.80,-17.13,72.72,43.14,230.0,-0.28,1.10,42.71,XY24,NTT,MISC
75,TITAGARH,1548.00,1.40,71.0,H-SC,4.95,166381.0,-25634.0,104521.0,2.51,-13.35,62.82,41.08,158.0,-0.25,1.21,40.99,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,43.43
2,50,73.85


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.43
LC    34.52
MC    23.04
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.93
X40      15.21
X40N     12.97
XR       10.04
XY25      9.78
AR        8.65
OX40N     8.17
X200      1.75
MH        1.74
X5K       1.47
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.41
H-LC    26.27
H-MC    20.08
M-SC    13.48
M-LC     7.18
M-MC     2.65
L-SC     1.54
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.34,-14.56,79.69
FMCG,11.84,-3.99,40.37
FINANCE,9.63,-15.97,61.39
BANKS,7.77,-12.25,63.93
MISC,6.04,-14.02,79.11
PAINTS,5.84,-14.81,40.51
ELECTRICAL,5.68,-5.27,63.29
AC,3.62,2.64,30.34
INSURANCE,3.40,-7.49,44.36


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2862681.0,22
AR,1305223.0,9
XR,1233040.0,13
X40,873472.0,10
X40N,627543.0,10
OX40N,522257.0,10
XY25,405610.0,7
SR,244974.0,2
X5K,114844.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3342533.0,28
M-SC,1387851.0,17
H-LC,1379294.0,15
H-MC,1189830.0,15
M-LC,390900.0,5
M-MC,312987.0,2
L-SC,249597.0,3
L-MC,60405.0,1
L-LC,37354.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1033383.0      6
M-SC       XY24       799443.0      7
H-SC       AR         785965.0      5
           XR         674260.0      6
H-LC       X40        587940.0      5
H-MC       XY24       489810.0      4
H-LC       AR         357113.0      2
M-MC       XY24       312987.0      2
H-SC       X40N       295105.0      4
           OX40N      247096.0      4
           SR         244974.0      2
H-LC       X40N       224985.0      4
H-MC       X40        215393.0      4
           XY25       185281.0      2
L-SC       XR         165766.0      2
M-SC       AR         162145.0      2
M-LC       XY24       161691.0      2
M-SC       OX40N      125476.0      4
           XR         116835.0      2
M-LC       X5K        114844.0      1
M-SC       XY25       113813.0      1
H-MC       XR         111571.0      1
           X40N       107453.0      2
M-LC       XR         104203.0      1
H-LC       X200        84889.0      1
L-SC       OX40N       83831.0      1
M-SC       X40         70139.0      1
H-MC       OX40N       65854.0      1
H-LC       XY24        65367.0      1
H-SC       MH          61750.0      1
L-MC       XR          60405.0      1
H-LC       XY25        59000.0      2
L-LC       XY25        37354.0      1
H-MC       MH          14468.0      1
M-LC       XY25        10162.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
